In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import sqlalchemy
from sqlalchemy import create_engine # database connection
import sqlite3
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.display import display, clear_output
from sklearn.naive_bayes import MultinomialNB
from datetime import datetime
from sklearn.metrics import accuracy_score

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
# print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [2]:
# engine  = create_engine("sqlite:///../input/database.sqlite")
with sqlite3.connect('database.sqlite') as engine:
#    matches = pd.read_sql_query('SELECT * FROM Match where league_id = 1729 and season in ("2010/2011", "2011/2012", "2012/2013", "2013/2014", "2014/2015", "2015/2016");'
#                                          , engine)
    matches = pd.read_sql_query('SELECT * FROM Match where league_id = 1729 ;', engine)


matches = matches[matches.columns[:11]]
teams = pd.read_sql_query('SELECT * FROM Team;', engine)

# Add team names & tidy up
matches = pd.merge(left=matches, right=teams, how='left', left_on='home_team_api_id', right_on='team_api_id')
matches = matches.drop(['country_id','league_id', 'home_team_api_id', 'id_y', 'team_api_id', 'team_short_name'], axis=1)
matches.rename(columns={'id_x':'match_id', 'team_long_name':'home_team'}, inplace=True)
matches = pd.merge(left=matches, right=teams, how='left', left_on='away_team_api_id', right_on='team_api_id')
matches = matches.drop(['id', 'match_api_id', 'away_team_api_id','team_api_id', 'team_short_name'], axis=1)
matches.rename(columns={'id_x':'match_id', 'team_long_name':'away_team'}, inplace=True)

matches.tail(20)
# matches.info()

,match_id,season,stage,date,home_team_goal,away_team_goal,team_fifa_api_id_x,home_team,team_fifa_api_id_y,away_team
3020,4749,2015/2016,8,2015-10-04 00:00:00,3,0,1.0,Arsenal,11.0,Manchester United
3021,4750,2015/2016,8,2015-10-03 00:00:00,0,1,2.0,Aston Villa,1806.0,Stoke City
3022,4751,2015/2016,8,2015-10-03 00:00:00,1,1,1943.0,Bournemouth,1795.0,Watford
3023,4752,2015/2016,8,2015-10-03 00:00:00,1,3,5.0,Chelsea,17.0,Southampton
3024,4753,2015/2016,8,2015-10-03 00:00:00,2,0,1799.0,Crystal Palace,109.0,West Bromwich Albion
3025,4754,2015/2016,8,2015-10-04 00:00:00,1,1,7.0,Everton,9.0,Liverpool
3026,4755,2015/2016,8,2015-10-03 00:00:00,6,1,10.0,Manchester City,13.0,Newcastle United
3027,4756,2015/2016,8,2015-10-03 00:00:00,1,2,1792.0,Norwich City,95.0,Leicester City
3028,4757,2015/2016,8,2015-10-03 00:00:00,2,2,106.0,Sunderland,19.0,West Ham United
3029,4758,2015/2016,8,2015-10-04 00:00:00,2,2,1960.0,Swansea City,18.0,Tottenham Hotspur


In [3]:
def convert_date_to_code16(matches_data_16):
    date = []
    for i in matches_data_16['date']:
        date.append(i)
    x = date[0]
    d = datetime.strptime(x, "%d/%m/%y").date()
    dict2 = {}
    base = 42595
    for i in date:
        date_l = datetime.strptime(i, "%d/%m/%y").date()
        date_dif = date_l - d
        dict2[date_l] = str(base + date_dif.days)
    t = []
    for i in range(len(date)):
        t.append(dict2[datetime.strptime(date[i], '%d/%m/%y').date()])
    matches_data_16['date'] = list(map((lambda i: t[i]), range(len(t))))
    return matches_data_16

In [4]:
def convert_date_to_code17(matches_data_17):
    stage = []
    for i in matches_data_17['stage']:
        stage.append(i)
    date = []
    for i in matches_data_17['date']:
        date.append(i)
    dict_date = {}
    for i, j in zip(stage, date):
        dict_date[i] = datetime.strptime(j, "%d/%m/%y").date()
    dict2 = {}
    base = 42959
    for i in stage:
        if i == 1:
            dict2[i] = base
        else:
            date_dif = dict_date[i] - dict_date[i - 1]
            if date_dif.days < 13:
                dict2[i] = dict2[i - 1] + 7
            if date_dif.days >= 13:
                dict2[i] = dict2[i - 1] + 14
    t = []
    for i in stage:
        t.append(str(dict2[i]))
    matches_data_17['date'] = list(map((lambda i: t[i]), range(len(t))))
    return matches_data_17

In [5]:
def convert_team_name(last_season):
    for index, row in last_season.iterrows():
        if row['home_team'] == 'Man United':
            last_season.loc[index, 'home_team'] = 'Manchester United'
        if row['home_team'] == 'Man City':
            last_season.loc[index, 'home_team'] = 'Manchester City'
        if row['home_team'] == 'Leicester':
            last_season.loc[index, 'home_team'] = 'Leicester City'
        if row['home_team'] == 'West Brom':
            last_season.loc[index, 'home_team'] = 'West Bromwich Albion'
        if row['home_team'] == 'Newcastle':
            last_season.loc[index, 'home_team'] = 'Newcastle United'
        if row['home_team'] == 'Stoke':
            last_season.loc[index, 'home_team'] = 'Stoke City'
        if row['home_team'] == 'Swansea':
            last_season.loc[index, 'home_team'] = 'Swansea City'
        if row['home_team'] == 'West Ham':
            last_season.loc[index, 'home_team'] = 'West Ham United'
        if row['home_team'] == 'Tottenham':
            last_season.loc[index, 'home_team'] = 'Tottenham Hotspur'
        if row['home_team'] == 'Hull':
            last_season.loc[index, 'home_team'] = 'Hull City'
        if row['home_team'] == 'Brighton & Hove Albion':
            last_season.loc[index, 'home_team'] = 'Brighton'
        if row['home_team'] == 'Huddersfield Town':
            last_season.loc[index, 'home_team'] = 'Huddersfield'

        if row['away_team'] == 'Man United':
            last_season.loc[index, 'away_team'] = 'Manchester United'
        if row['away_team'] == 'Man City':
            last_season.loc[index, 'away_team'] = 'Manchester City'
        if row['away_team'] == 'Leicester':
            last_season.loc[index, 'away_team'] = 'Leicester City'
        if row['away_team'] == 'West Brom':
            last_season.loc[index, 'away_team'] = 'West Bromwich Albion'
        if row['away_team'] == 'Newcastle':
            last_season.loc[index, 'away_team'] = 'Newcastle United'
        if row['away_team'] == 'Stoke':
            last_season.loc[index, 'away_team'] = 'Stoke City'
        if row['away_team'] == 'Swansea':
            last_season.loc[index, 'away_team'] = 'Swansea City'
        if row['away_team'] == 'West Ham':
            last_season.loc[index, 'away_team'] = 'West Ham United'
        if row['away_team'] == 'Tottenham':
            last_season.loc[index, 'away_team'] = 'Tottenham Hotspur'
        if row['away_team'] == 'Hull':
            last_season.loc[index, 'away_team'] = 'Hull City'
        if row['away_team'] == 'Brighton & Hove Albion':
            last_season.loc[index, 'away_team'] = 'Brighton'
        if row['away_team'] == 'Huddersfield Town':
            last_season.loc[index, 'away_team'] = 'Huddersfield'
    return last_season

In [6]:
def create_match_id():
    match_id_num = []
    match_id_num2 = []
    for i in range(1, 11):
        match_id_num.append(i)
    for i in match_id_num:
        match_id_num2.append(i + int(last_season[-1:]['match_id']))
    return match_id_num2

In [7]:
with sqlite3.connect('database2.sqlite') as con:
    matches_data_16 = pd.read_sql('select * from match;', con)
    matches_data_17 = pd.read_sql('select * from match17;', con)
    matches_schedule = pd.read_sql('select * from match_schedule;', con)
    team_rating = pd.read_sql('select * from team_rating;', con)
# matches_data_16['date'] = pd.to_datetime(matches_data_16['date'])

matches_data_16 = convert_date_to_code16(matches_data_16)
matches_data_17 = convert_date_to_code17(matches_data_17)
matches_schedule = convert_date_to_code17(matches_schedule)
last_season = pd.concat([matches_data_16, matches_data_17])
# int(last_season[-1:]['match_id'])
# matches_schedule.set_index('index', inplace=True)
# matches_schedule.columns = ['date', 'home_team_goal', 'away_team_goal', 'home_team', 'away_team', 'season', 'stage']
# matches_schedule = convert_team_name(matches_schedule)
# matches_schedule.head(190)
match_id = create_match_id()
# match_id
last_season.tail()

,match_id,season,stage,date,home_team_goal,away_team_goal,home_team,away_team
274,7274,2017/2018,28,43176,4,1,Liverpool,West Ham
275,7275,2017/2018,28,43176,1,0,Watford,Everton
276,7276,2017/2018,28,43176,1,2,West Brom,Huddersfield
277,7277,2017/2018,28,43176,0,1,Crystal Palace,Tottenham
278,7278,2017/2018,28,43176,2,1,Man United,Chelsea


In [8]:
# Parameters to change depending on season and wek we are running for
this_season = '2017/2018'
this_week = int(matches_data_17[-1:]['stage']) + 1
train_ratio = .8
model_weight = .8
this_week

29

In [9]:
matches_schedule = matches_schedule.loc[matches_schedule['stage'] == this_week]
matches_schedule['match_id'] = match_id
matches_schedule

,match_id,date,home_team,away_team,season,stage
280,7279,43183,Brighton & Hove Albion,Arsenal,2017/2018,29
281,7280,43183,Burnley,Everton,2017/2018,29
282,7281,43183,Crystal Palace,Manchester United,2017/2018,29
283,7282,43183,Leicester City,Bournemouth,2017/2018,29
284,7283,43183,Liverpool,Newcastle United,2017/2018,29
285,7284,43183,Manchester City,Chelsea,2017/2018,29
286,7285,43183,Southampton,Stoke City,2017/2018,29
287,7286,43183,Swansea,West Ham United,2017/2018,29
288,7287,43183,Tottenham Hotspur,Huddersfield Town,2017/2018,29
289,7288,43183,Watford,West Bromwich Albion,2017/2018,29


In [10]:
# Add to full training data to predict current season
matches = pd.concat([matches, last_season, matches_schedule])
matches = matches.reset_index(drop=True)
convert_team_name(matches)
# matches

# Create a full set of match data that can be used with feature engineering later
full_matches = matches.copy()
# full_matches.match_id.fillna(last_season['match_id'][-1:] + 1)
full_matches

,away_team,away_team_goal,date,home_team,home_team_goal,match_id,season,stage,team_fifa_api_id_x,team_fifa_api_id_y
0,Newcastle United,1.0,2008-08-17 00:00:00,Manchester United,1.0,1729,2008/2009,1,11.0,13.0
1,West Bromwich Albion,0.0,2008-08-16 00:00:00,Arsenal,1.0,1730,2008/2009,1,1.0,109.0
2,Liverpool,1.0,2008-08-16 00:00:00,Sunderland,0.0,1731,2008/2009,1,106.0,9.0
3,Wigan Athletic,1.0,2008-08-16 00:00:00,West Ham United,2.0,1732,2008/2009,1,19.0,1917.0
4,Manchester City,2.0,2008-08-17 00:00:00,Aston Villa,4.0,1733,2008/2009,1,2.0,10.0
5,Blackburn Rovers,3.0,2008-08-16 00:00:00,Everton,2.0,1734,2008/2009,1,7.0,3.0
6,Tottenham Hotspur,1.0,2008-08-16 00:00:00,Middlesbrough,2.0,1735,2008/2009,1,12.0,18.0
7,Stoke City,1.0,2008-08-16 00:00:00,Bolton Wanderers,3.0,1736,2008/2009,1,4.0,1806.0
8,Fulham,1.0,2008-08-16 00:00:00,Hull City,2.0,1737,2008/2009,1,1952.0,144.0
9,Portsmouth,0.0,2008-08-17 00:00:00,Chelsea,4.0,1738,2008/2009,1,5.0,1790.0


In [11]:
full_matches.drop(matches[matches.season == '2008/2009'].index, inplace=True)
full_matches.drop(matches[matches.season == '2009/2010'].index, inplace=True)
full_matches.drop(matches[matches.season == '2010/2011'].index, inplace=True)
full_matches.drop(matches[matches.season == '2011/2012'].index, inplace=True)

full_matches

,away_team,away_team_goal,date,home_team,home_team_goal,match_id,season,stage,team_fifa_api_id_x,team_fifa_api_id_y
1520,Sunderland,0.0,2012-08-18 00:00:00,Arsenal,0.0,3249,2012/2013,1,1.0,106.0
1521,Manchester United,0.0,2012-08-20 00:00:00,Everton,1.0,3250,2012/2013,1,7.0,11.0
1522,Norwich City,0.0,2012-08-18 00:00:00,Fulham,5.0,3251,2012/2013,1,144.0,1792.0
1523,Southampton,2.0,2012-08-19 00:00:00,Manchester City,3.0,3252,2012/2013,1,10.0,17.0
1524,Tottenham Hotspur,1.0,2012-08-18 00:00:00,Newcastle United,2.0,3253,2012/2013,1,13.0,18.0
1525,Swansea City,5.0,2012-08-18 00:00:00,Queens Park Rangers,0.0,3254,2012/2013,1,15.0,1960.0
1526,Stoke City,1.0,2012-08-18 00:00:00,Reading,1.0,3255,2012/2013,1,1793.0,1806.0
1527,Liverpool,0.0,2012-08-18 00:00:00,West Bromwich Albion,3.0,3256,2012/2013,1,109.0,9.0
1528,Aston Villa,0.0,2012-08-18 00:00:00,West Ham United,1.0,3257,2012/2013,1,19.0,2.0
1529,Chelsea,2.0,2012-08-19 00:00:00,Wigan Athletic,0.0,3258,2012/2013,1,1917.0,5.0


full_matches : ข้อมูลการแข่งขันทั้งหมดรวมอาทิตย์ต่อไปด้วย

In [12]:
# State the result as home or away win/draw/lose - 6 possibilites
# Create a binary result
def determine_result(match_list):
    match_list['home_win'] = np.where(match_list['home_team_goal'] > match_list['away_team_goal'], 1, 0)
    match_list['home_draw'] = np.where(match_list['home_team_goal'] == match_list['away_team_goal'], 1, 0)
    match_list['home_lose'] = np.where(match_list['home_team_goal'] < match_list['away_team_goal'], 1, 0)
    match_list['away_win'] = np.where(match_list['home_team_goal'] < match_list['away_team_goal'], 1, 0)
    match_list['away_draw'] = np.where(match_list['home_team_goal'] == match_list['away_team_goal'], 1, 0)
    match_list['away_lose'] = np.where(match_list['home_team_goal'] > match_list['away_team_goal'], 1, 0)

# Set up the matches data how I need it

# Add binary feature for W/D/L home and away
determine_result(full_matches)

# Sort in date order
full_matches.sort_values(by='date', inplace=True)

full_matches.head()

,away_team,away_team_goal,date,home_team,home_team_goal,match_id,season,stage,team_fifa_api_id_x,team_fifa_api_id_y,home_win,home_draw,home_lose,away_win,away_draw,away_lose
1520,Sunderland,0.0,2012-08-18 00:00:00,Arsenal,0.0,3249,2012/2013,1,1.0,106.0,0,1,0,0,1,0
1527,Liverpool,0.0,2012-08-18 00:00:00,West Bromwich Albion,3.0,3256,2012/2013,1,109.0,9.0,1,0,0,0,0,1
1526,Stoke City,1.0,2012-08-18 00:00:00,Reading,1.0,3255,2012/2013,1,1793.0,1806.0,0,1,0,0,1,0
1525,Swansea City,5.0,2012-08-18 00:00:00,Queens Park Rangers,0.0,3254,2012/2013,1,15.0,1960.0,0,0,1,1,0,0
1528,Aston Villa,0.0,2012-08-18 00:00:00,West Ham United,1.0,3257,2012/2013,1,19.0,2.0,1,0,0,0,0,1


new_team : ข้อมูลทีมที่เลื่อนชั้น พร้อมฤดูกาล

In [13]:
# Cope with newly promoted teams with limited or no stats
team_data = {'team':['West Bromwich Albion', 'Stoke City', 'Hull City',
                     'Wolverhampton Wanderers', 'Birmingham City', 'Burnley',
                     'Newcastle United', 'West Bromwich Albion', 'Blackpool',
                     'Queens Park Rangers', 'Norwich City', 'Swansea City',
                     'Reading', 'Southampton', 'West Ham United',
                     'Cardiff City', 'Crystal Palace', 'Hull City',
                     'Leicester City', 'Burnley', 'Queens Park Rangers',
                     'Bournemouth', 'Watford', 'Norwich City',
                     'Burnley', 'Middlesbrough', 'Hull City',
                     'Brighton', 'Newcastle United', 'Huddersfield'
                    ],
             'season':["2008/2009", "2008/2009", "2008/2009",
                       "2009/2010", "2009/2010", "2009/2010",
                       "2010/2011", "2010/2011", "2010/2011",
                       "2011/2012", "2011/2012", "2011/2012",
                       "2012/2013", "2012/2013", "2012/2013",
                       "2013/2014", "2013/2014", "2013/2014",
                       "2014/2015", "2014/2015", "2014/2015",
                       "2015/2016", "2015/2016", "2015/2016",
                       "2016/2017", "2016/2017", "2016/2017",
                       "2017/2018", "2017/2018", "2017/2018"
                      ]
            }
new_teams = pd.DataFrame(team_data, columns=['team', 'season'])
new_teams.head()

,team,season
0,West Bromwich Albion,2008/2009
1,Stoke City,2008/2009
2,Hull City,2008/2009
3,Wolverhampton Wanderers,2009/2010
4,Birmingham City,2009/2010


full_match_features : dummy ข้อมูลของสถิติเก่าๆ

In [14]:
full_match_features = pd.DataFrame(full_matches[['season', 'stage']])
                                   #,
                                   #columns=[['season', 'stage']])
full_match_features.head()

,season,stage
1520,2012/2013,1
1527,2012/2013,1
1526,2012/2013,1
1525,2012/2013,1
1528,2012/2013,1


In [15]:
team_rating

,team_rating_id,season,team_name,attack_rating,midfield_rating,defence_rating,team_rating
0,0,2012/2013,Manchester United,86,81,82,82
1,1,2012/2013,Manchester City,85,83,80,82
2,2,2012/2013,Chelsea,82,80,81,81
3,3,2012/2013,Tottenham Hotspur,82,78,79,80
4,4,2012/2013,Arsenal,80,79,80,80
5,5,2012/2013,Liverpool,76,80,79,79
6,6,2012/2013,Newcastle United,81,77,77,77
7,7,2012/2013,Fulham,78,75,75,77
8,8,2012/2013,Everton,74,76,77,76
9,9,2012/2013,Queens Park Rangers,77,73,74,75


In [16]:
team_rating.drop('team_rating_id', axis=1, inplace=True)
team_rating2 = team_rating.copy()
team_rating = team_rating.rename(columns = {'team_name': 'home_team'})
team_rating2 = team_rating2.rename(columns = {'team_name': 'away_team'})

In [17]:
full_matches = pd.merge(full_matches, team_rating, how='left', on=['season', 'home_team'])
full_matches = full_matches.rename(columns = {'attack_rating': 'home_team_attack_rating'})
full_matches = full_matches.rename(columns = {'midfield_rating': 'home_team_midfield_rating'})
full_matches = full_matches.rename(columns = {'defence_rating': 'home_team_defence_rating'})
full_matches = full_matches.rename(columns = {'team_rating': 'home_team_rating'})
full_matches = pd.merge(full_matches, team_rating2, how='left', on=['season', 'away_team'])
full_matches = full_matches.rename(columns = {'attack_rating': 'away_team_attack_rating'})
full_matches = full_matches.rename(columns = {'midfield_rating': 'away_team_midfield_rating'})
full_matches = full_matches.rename(columns = {'defence_rating': 'away_team_defence_rating'})
full_matches = full_matches.rename(columns = {'team_rating': 'away_team_rating'})
full_matches

,away_team,away_team_goal,date,home_team,home_team_goal,match_id,season,stage,team_fifa_api_id_x,team_fifa_api_id_y,...,away_draw,away_lose,home_team_attack_rating,home_team_midfield_rating,home_team_defence_rating,home_team_rating,away_team_attack_rating,away_team_midfield_rating,away_team_defence_rating,away_team_rating
0,Sunderland,0.0,2012-08-18 00:00:00,Arsenal,0.0,3249,2012/2013,1,1.0,106.0,...,1,0,80,79,80,80,75,73,74,75
1,Liverpool,0.0,2012-08-18 00:00:00,West Bromwich Albion,3.0,3256,2012/2013,1,109.0,9.0,...,0,1,74,74,71,74,76,80,79,79
2,Stoke City,1.0,2012-08-18 00:00:00,Reading,1.0,3255,2012/2013,1,1793.0,1806.0,...,1,0,72,69,71,71,74,74,75,75
3,Swansea City,5.0,2012-08-18 00:00:00,Queens Park Rangers,0.0,3254,2012/2013,1,15.0,1960.0,...,0,0,77,73,74,75,74,73,73,74
4,Aston Villa,0.0,2012-08-18 00:00:00,West Ham United,1.0,3257,2012/2013,1,19.0,2.0,...,0,1,75,74,72,74,74,72,71,75
5,Norwich City,0.0,2012-08-18 00:00:00,Fulham,5.0,3251,2012/2013,1,144.0,1792.0,...,0,1,78,75,75,77,72,73,73,73
6,Tottenham Hotspur,1.0,2012-08-18 00:00:00,Newcastle United,2.0,3253,2012/2013,1,13.0,18.0,...,0,1,81,77,77,77,82,78,79,80
7,Southampton,2.0,2012-08-19 00:00:00,Manchester City,3.0,3252,2012/2013,1,10.0,17.0,...,0,1,85,83,80,82,73,74,71,73
8,Chelsea,2.0,2012-08-19 00:00:00,Wigan Athletic,0.0,3258,2012/2013,1,1917.0,5.0,...,0,0,74,72,72,74,82,80,81,81
9,Manchester United,0.0,2012-08-20 00:00:00,Everton,1.0,3250,2012/2013,1,7.0,11.0,...,0,1,74,76,77,76,86,81,82,82


In [18]:
# Convert home & team into a binary feature, ie Arsenal_h or Arsenal_a
# Need all seasons data for team binary feature
full_match_features = pd.DataFrame(full_matches[['season', 'stage']]) # เอา column ไหนมาบ้าง
#                                    columns=[['season', 'stage']])

full_match_features = pd.concat([full_match_features, pd.get_dummies(full_matches['home_team']).rename(columns=lambda x: str(x) + '_h')],
                                axis=1)
full_match_features = pd.concat([full_match_features, pd.get_dummies(full_matches['away_team']).rename(columns=lambda x: str(x) + '_a')],
                         axis=1)

full_match_features.head() # ข้อมูลสถิติเก่าๆ

,season,stage,Arsenal_h,Aston Villa_h,Bournemouth_h,Brighton_h,Burnley_h,Cardiff City_h,Chelsea_h,Crystal Palace_h,...,Reading_a,Southampton_a,Stoke City_a,Sunderland_a,Swansea City_a,Tottenham Hotspur_a,Watford_a,West Bromwich Albion_a,West Ham United_a,Wigan Athletic_a
0,2012/2013,1,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,2012/2013,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2012/2013,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,2012/2013,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,2012/2013,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


full_match_features_2 : ข้อมูล rating team เก่าๆ

In [19]:
# DataFrame team rating
full_match_features_2 = pd.DataFrame(full_matches[['season', 'stage', 'home_team_attack_rating',
       'home_team_midfield_rating', 'home_team_defence_rating',
       'home_team_rating', 'away_team_attack_rating',
       'away_team_midfield_rating', 'away_team_defence_rating',
       'away_team_rating']])

full_match_features_2.head() # ข้อมูล team rating

,season,stage,home_team_attack_rating,home_team_midfield_rating,home_team_defence_rating,home_team_rating,away_team_attack_rating,away_team_midfield_rating,away_team_defence_rating,away_team_rating
0,2012/2013,1,80,79,80,80,75,73,74,75
1,2012/2013,1,74,74,71,74,76,80,79,79
2,2012/2013,1,72,69,71,71,74,74,75,75
3,2012/2013,1,77,73,74,75,74,73,73,74
4,2012/2013,1,75,74,72,74,74,72,71,75


In [20]:
# Function to determine whether the highest prediction is for win/draw/lose

def predict_home_result(match):
    if (match['win'] >= match['draw']) & (match['win'] >= match['lose']):
        return 'win' # Favour a home win if probability equal
    elif (match['lose'] > match['win']) & (match['lose'] > match['draw']):
        return 'lose'
    else:
        return 'draw'

train_match_features : ข้อมูล dummy สถิติเก่าๆ ไว้ train model 

In [21]:
# To predict this season (1, this week only, remove this week from training set
train_match_features = full_match_features.loc[:int(full_match_features.shape[0] * train_ratio )].copy()

train_match_features.drop(['season'], axis=1, inplace=True)
train_match_features.tail()

,stage,Arsenal_h,Aston Villa_h,Bournemouth_h,Brighton_h,Burnley_h,Cardiff City_h,Chelsea_h,Crystal Palace_h,Everton_h,...,Reading_a,Southampton_a,Stoke City_a,Sunderland_a,Swansea City_a,Tottenham Hotspur_a,Watford_a,West Bromwich Albion_a,West Ham United_a,Wigan Athletic_a
1747,23,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1748,23,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1749,23,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1750,24,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1751,24,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


train_match_features_2 : ข้อมูล rating team เก่าๆ ไว้ train model

In [22]:
# To predict this season (1, this week only, remove this week from training set
train_match_features_2 = full_match_features_2.loc[:int(full_match_features.shape[0] * train_ratio  )].copy()

train_match_features_2.drop(['season', 'stage'], axis=1, inplace=True)
train_match_features_2.tail()

,home_team_attack_rating,home_team_midfield_rating,home_team_defence_rating,home_team_rating,away_team_attack_rating,away_team_midfield_rating,away_team_defence_rating,away_team_rating
1747,84,84,82,83,76,74,75,75
1748,77,78,77,78,85,78,79,80
1749,79,77,77,78,83,85,81,83
1750,81,76,74,75,77,78,77,78
1751,86,84,81,83,81,83,82,83


In [23]:
# Get the target results for training

# Function to determine whether result is a win/draw/lose
def determine_home_result(match):
    if match['home_team_goal'] > match['away_team_goal']:
        return 'win'
    elif match['home_team_goal'] < match['away_team_goal']:
        return 'lose'
    else:
        return 'draw'

train_matches : ข้อมูล train ทั้งหมดพร้อม label แล้ว

In [24]:
#  Add the home team result column to the matches dataframe
full_matches['home_team_result'] = full_matches.apply(determine_home_result, axis=1)

# To predict this season, this week, remove latest week from training results
train_matches = full_matches.loc[:int(full_matches.shape[0] *  train_ratio  )].copy()

targets = train_matches['home_team_result'].values
train_matches.tail()

,away_team,away_team_goal,date,home_team,home_team_goal,match_id,season,stage,team_fifa_api_id_x,team_fifa_api_id_y,...,away_lose,home_team_attack_rating,home_team_midfield_rating,home_team_defence_rating,home_team_rating,away_team_attack_rating,away_team_midfield_rating,away_team_defence_rating,away_team_rating,home_team_result
1747,Hull City,0.0,42767,Manchester United,0.0,6227,2016/2017,23,NaN,NaN,...,0,84,84,82,83,76,74,75,75,draw
1748,Everton,1.0,42767,Stoke City,1.0,6228,2016/2017,23,NaN,NaN,...,0,77,78,77,78,85,78,79,80,draw
1749,Manchester City,4.0,42767,West Ham United,0.0,6229,2016/2017,23,NaN,NaN,...,0,79,77,77,78,83,85,81,83,lose
1750,Stoke City,0.0,42770,West Bromwich Albion,1.0,6237,2016/2017,24,NaN,NaN,...,1,81,76,74,75,77,78,77,78,win
1751,Arsenal,1.0,42770,Chelsea,3.0,6230,2016/2017,24,NaN,NaN,...,1,86,84,81,83,81,83,82,83,win


test_match_features : ข้อมูล dummy สถิติเก่าๆ ไว้ test model

In [25]:
# Get the test matches in correct format:

# Predict this season this week
test_match_features = full_match_features.loc[int(full_match_features.shape[0] * train_ratio  ):].copy()

test_match_features.drop(full_match_features.index[-10:], inplace=True)
test_match_features.drop(['season'], axis=1, inplace=True)
test_match_features

,stage,Arsenal_h,Aston Villa_h,Bournemouth_h,Brighton_h,Burnley_h,Cardiff City_h,Chelsea_h,Crystal Palace_h,Everton_h,...,Reading_a,Southampton_a,Stoke City_a,Sunderland_a,Swansea City_a,Tottenham Hotspur_a,Watford_a,West Bromwich Albion_a,West Ham United_a,Wigan Athletic_a
1751,24,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1752,24,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
1753,24,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1754,24,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1755,24,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1756,24,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1757,24,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1758,24,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1759,24,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1760,25,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


test_match_features_1 : ข้อมูล dummy ของ week ถัดไป ไว้ทำนายจาก model สถิติเก่า

In [26]:
test_match_features_1 = full_match_features.iloc[-10:].copy()
test_match_features_1.drop(['season'], axis=1, inplace=True)
test_match_features_1

,stage,Arsenal_h,Aston Villa_h,Bournemouth_h,Brighton_h,Burnley_h,Cardiff City_h,Chelsea_h,Crystal Palace_h,Everton_h,...,Reading_a,Southampton_a,Stoke City_a,Sunderland_a,Swansea City_a,Tottenham Hotspur_a,Watford_a,West Bromwich Albion_a,West Ham United_a,Wigan Athletic_a
2179,29,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2180,29,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2181,29,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2182,29,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2183,29,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2184,29,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2185,29,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2186,29,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2187,29,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2188,29,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


test_match_features_2 : ข้อมูล team rating เก่าๆไว้ test model

In [27]:
test_match_features_2 = full_match_features_2.loc[int(full_match_features.shape[0] * train_ratio  ):].copy()

test_match_features_2.drop(full_match_features.index[-10:], inplace=True)
test_match_features_2.drop(['season', 'stage'], axis=1, inplace=True)
test_match_features_2

,home_team_attack_rating,home_team_midfield_rating,home_team_defence_rating,home_team_rating,away_team_attack_rating,away_team_midfield_rating,away_team_defence_rating,away_team_rating
1751,86,84,81,83,81,83,82,83
1752,80,77,76,77,81,76,73,76
1753,85,78,79,80,74,75,74,74
1754,76,74,75,75,82,81,81,81
1755,78,76,77,78,79,77,77,78
1756,85,81,81,81,76,74,74,75
1757,78,78,76,77,74,74,76,74
1758,83,85,81,83,78,76,75,77
1759,79,78,76,78,84,84,82,83
1760,81,83,82,83,76,74,75,75


test_match_features_2_1 : ข้อมูล next week ของ team rating ไว้ทำนายจาก model team rating

In [28]:
test_match_features_2_1 = full_match_features_2.iloc[-10:].copy()
test_match_features_2_1.drop(['season', 'stage'], axis=1, inplace=True)
test_match_features_2_1

,home_team_attack_rating,home_team_midfield_rating,home_team_defence_rating,home_team_rating,away_team_attack_rating,away_team_midfield_rating,away_team_defence_rating,away_team_rating
2179,85,87,82,84,85,84,83,84
2180,76,75,74,75,79,79,77,78
2181,78,77,77,77,77,77,76,77
2182,84,81,79,81,76,75,75,75
2183,85,83,82,83,75,73,73,73
2184,81,76,75,76,86,84,81,84
2185,74,76,76,76,80,80,77,80
2186,74,75,74,75,84,82,81,82
2187,79,79,76,78,77,74,75,75
2188,79,77,75,77,78,77,77,77


model_test_matches : ข้อมูลทั้งหมดไว้ test model

In [29]:
# Don't have the target results yet but I have entered dummy data of 0-0 draws. 
# Then re-run with actual scores after they are played for comparison
model_test_matches = full_matches.loc[int(full_matches.shape[0] * train_ratio  ):].copy()
# model_test_matches = full_matches.loc[(full_matches['season'] == this_season) & 
#                                      (full_matches['stage'] == this_week - 1)].copy()

model_test_matches.drop(full_match_features.index[-10:], inplace=True)
model_test_matches = model_test_matches.reset_index(drop=True)
model_test_matches

,away_team,away_team_goal,date,home_team,home_team_goal,match_id,season,stage,team_fifa_api_id_x,team_fifa_api_id_y,...,away_lose,home_team_attack_rating,home_team_midfield_rating,home_team_defence_rating,home_team_rating,away_team_attack_rating,away_team_midfield_rating,away_team_defence_rating,away_team_rating,home_team_result
0,Arsenal,1.0,42770,Chelsea,3.0,6230,2016/2017,24,NaN,NaN,...,1,86,84,81,83,81,83,82,83,win
1,Sunderland,4.0,42770,Crystal Palace,0.0,6231,2016/2017,24,NaN,NaN,...,0,80,77,76,77,81,76,73,76,lose
2,Bournemouth,3.0,42770,Everton,6.0,6232,2016/2017,24,NaN,NaN,...,1,85,78,79,80,74,75,74,74,win
3,Liverpool,0.0,42770,Hull City,2.0,6233,2016/2017,24,NaN,NaN,...,1,76,74,75,75,82,81,81,81,win
4,West Ham United,3.0,42770,Southampton,1.0,6234,2016/2017,24,NaN,NaN,...,0,78,76,77,78,79,77,77,78,lose
5,Middlesbrough,0.0,42770,Tottenham Hotspur,1.0,6235,2016/2017,24,NaN,NaN,...,1,85,81,81,81,76,74,74,75,win
6,Burnley,1.0,42770,Watford,2.0,6236,2016/2017,24,NaN,NaN,...,1,78,78,76,77,74,74,76,74,win
7,Swansea City,1.0,42771,Manchester City,2.0,6239,2016/2017,24,NaN,NaN,...,1,83,85,81,83,78,76,75,77,win
8,Manchester United,3.0,42771,Leicester City,0.0,6238,2016/2017,24,NaN,NaN,...,0,79,78,76,78,84,84,82,83,lose
9,Hull City,0.0,42777,Arsenal,2.0,6240,2016/2017,25,NaN,NaN,...,1,81,83,82,83,76,74,75,75,win


model_test_matches_1 : ข้อมูลทั้งหมดของ next week

In [30]:
model_test_matches_1 = full_matches.iloc[-10:].copy()
# model_test_matches = full_matches.loc[(full_matches['season'] == this_season) & 
#                                      (full_matches['stage'] == this_week - 1)].copy()

# model_test_matches.drop(full_match_features.index[-10:], inplace=True)
model_test_matches_1 = model_test_matches_1.reset_index(drop=True)
model_test_matches_1

,away_team,away_team_goal,date,home_team,home_team_goal,match_id,season,stage,team_fifa_api_id_x,team_fifa_api_id_y,...,away_lose,home_team_attack_rating,home_team_midfield_rating,home_team_defence_rating,home_team_rating,away_team_attack_rating,away_team_midfield_rating,away_team_defence_rating,away_team_rating,home_team_result
0,Chelsea,NaN,43183,Manchester City,NaN,7284,2017/2018,29,NaN,NaN,...,0,85,87,82,84,85,84,83,84,draw
1,West Ham United,NaN,43183,Swansea City,NaN,7286,2017/2018,29,NaN,NaN,...,0,76,75,74,75,79,79,77,78,draw
2,Stoke City,NaN,43183,Southampton,NaN,7285,2017/2018,29,NaN,NaN,...,0,78,77,77,77,77,77,76,77,draw
3,Newcastle United,NaN,43183,Liverpool,NaN,7283,2017/2018,29,NaN,NaN,...,0,84,81,79,81,76,75,75,75,draw
4,Huddersfield,NaN,43183,Tottenham Hotspur,NaN,7287,2017/2018,29,NaN,NaN,...,0,85,83,82,83,75,73,73,73,draw
5,Manchester United,NaN,43183,Crystal Palace,NaN,7281,2017/2018,29,NaN,NaN,...,0,81,76,75,76,86,84,81,84,draw
6,Everton,NaN,43183,Burnley,NaN,7280,2017/2018,29,NaN,NaN,...,0,74,76,76,76,80,80,77,80,draw
7,Arsenal,NaN,43183,Brighton,NaN,7279,2017/2018,29,NaN,NaN,...,0,74,75,74,75,84,82,81,82,draw
8,Bournemouth,NaN,43183,Leicester City,NaN,7282,2017/2018,29,NaN,NaN,...,0,79,79,76,78,77,74,75,75,draw
9,West Bromwich Albion,NaN,43183,Watford,NaN,7288,2017/2018,29,NaN,NaN,...,0,79,77,75,77,78,77,77,77,draw


## Fit model

fit ของ ข้อมูลการแข่งขันเก่าๆ

In [31]:
# Train, then predict 
model = MultinomialNB()

model.fit(train_match_features.values, targets) # รับเป็น array ของสิ่งที่จะทำนายและผลเข้าไป
predicted = model.predict_proba(test_match_features.values) # รับเป็น array อาทิตที่จะทำนายต่อไป

# Format the output into a DF with columns
predicted_table = pd.DataFrame(predicted,columns=['draw', 'lose', 'win'])

# Compare predicted with test actual results 
predicted_table['predict_res'] = predicted_table.apply(predict_home_result, axis=1)
predicted_table['actual_res'] = model_test_matches['home_team_result']

# Straight comparison - count of equal / total to get %
(predicted_table[predicted_table['predict_res'] 
                 == model_test_matches['home_team_result']].count()) / model_test_matches['home_team_result'].count()

draw           0.584112
lose           0.584112
win            0.584112
predict_res    0.584112
actual_res     0.584112
dtype: float64

test model ข้อมูลการแข่งขันเก่าๆ

In [32]:
predicted_1 = model.predict_proba(test_match_features_1.values) # รับเป็น array อาทิตที่จะทำนายต่อไป

# Format the output into a DF with columns
predicted_table_1 = pd.DataFrame(predicted_1,columns=['draw', 'lose', 'win'])

# Compare predicted with test actual results 
predicted_table_1['predict_res'] = predicted_table_1.apply(predict_home_result, axis=1)
predicted_table_1['actual_res'] = model_test_matches_1['home_team_result']

# Straight comparison - count of equal / total to get %
(predicted_table_1[predicted_table_1['predict_res'] 
                 == model_test_matches_1['home_team_result']].count()) / model_test_matches_1['home_team_result'].count()

draw           0.0
lose           0.0
win            0.0
predict_res    0.0
actual_res     0.0
dtype: float64

fit ข้อมูล team rating เก่าๆ

In [33]:
# Train, then predict 
model_2 = MultinomialNB()

model_2.fit(train_match_features_2.values, targets) # รับเป็น array ของสิ่งที่จะทำนายและผลเข้าไป
predicted_2 = model_2.predict_proba(test_match_features_2.values) # รับเป็น array อาทิตที่จะทำนายต่อไป

# Format the output into a DF with columns
predicted_table_2 = pd.DataFrame(predicted_2,columns=['draw', 'lose', 'win'])

# Compare predicted with test actual results 
predicted_table_2['predict_res'] = predicted_table_2.apply(predict_home_result, axis=1)
predicted_table_2['actual_res'] = model_test_matches['home_team_result']

# Straight comparison - count of equal / total to get %
(predicted_table_2[predicted_table_2['predict_res'] 
                 == model_test_matches['home_team_result']].count()) / model_test_matches['home_team_result'].count()

draw           0.577103
lose           0.577103
win            0.577103
predict_res    0.577103
actual_res     0.577103
dtype: float64

train model ข้อมูล team rating

In [34]:
predicted_2_1 = model_2.predict_proba(test_match_features_2_1.values) # รับเป็น array อาทิตที่จะทำนายต่อไป

# Format the output into a DF with columns
predicted_table_2_1 = pd.DataFrame(predicted_2_1,columns=['draw', 'lose', 'win'])

# Compare predicted with test actual results 
predicted_table_2_1['predict_res'] = predicted_table_2_1.apply(predict_home_result, axis=1)
predicted_table_2_1['actual_res'] = model_test_matches_1['home_team_result']

# Straight comparison - count of equal / total to get %
(predicted_table_2_1[predicted_table_2_1['predict_res'] 
                 == model_test_matches_1['home_team_result']].count()) / model_test_matches_1['home_team_result'].count()

draw           0.0
lose           0.0
win            0.0
predict_res    0.0
actual_res     0.0
dtype: float64

## Test data result from old match

In [35]:
compare_results = model_test_matches[['match_id', 'stage', 'home_team_goal', 
                                    'away_team_goal', 'home_team', 'away_team']].copy()
compare_results.rename(columns={'home_team_goal':'h_goal', 'away_team_goal':'a_goal'}, inplace=True)
compare_results = pd.concat([compare_results, predicted_table], axis=1)
compare_results = compare_results[['match_id', 'stage', 'h_goal', 'a_goal', 'home_team', 'predict_res', 'away_team','draw', 'lose', 'win', 'actual_res']]
compare_results = compare_results.rename(columns={'predict_res': 'predict_result'})
compare_results.tail(10)

,match_id,stage,h_goal,a_goal,home_team,predict_result,away_team,draw,lose,win,actual_res
418,7264,27,1.0,0.0,Tottenham Hotspur,win,Arsenal,0.221593,0.380557,0.397850,win
419,7278,28,2.0,1.0,Manchester United,win,Chelsea,0.181555,0.373279,0.445166,win
420,7277,28,0.0,1.0,Crystal Palace,lose,Tottenham Hotspur,0.131593,0.699388,0.169019,lose
421,7276,28,1.0,2.0,West Bromwich Albion,lose,Huddersfield,0.324261,0.381288,0.294451,lose
422,7275,28,1.0,0.0,Watford,draw,Everton,0.375292,0.349069,0.275639,win
423,7273,28,1.0,1.0,Leicester City,win,Stoke City,0.309222,0.162323,0.528455,draw
424,7272,28,1.0,1.0,Burnley,win,Southampton,0.243366,0.335351,0.421283,draw
425,7271,28,4.0,1.0,Brighton,win,Swansea City,0.261419,0.315989,0.422592,win
426,7270,28,2.0,2.0,Bournemouth,win,Newcastle United,0.208859,0.279807,0.511334,draw
427,7274,28,4.0,1.0,Liverpool,win,West Ham United,0.251290,0.121194,0.627516,win


## Test data result from old team rating

In [36]:
compare_results_2 = model_test_matches[['match_id', 'stage', 'home_team_goal', 
                                    'away_team_goal', 'home_team', 'away_team']].copy()
compare_results_2.rename(columns={'home_team_goal':'h_goal', 'away_team_goal':'a_goal'}, inplace=True)
compare_results_2 = pd.concat([compare_results_2, predicted_table_2], axis=1)
compare_results_2 = compare_results_2[['match_id', 'stage', 'h_goal', 'a_goal', 'home_team', 'predict_res', 'away_team','draw', 'lose', 'win', 'actual_res']]
compare_results_2 = compare_results_2.rename(columns={'predict_res': 'predict_result'})
compare_results_2.tail(10)

,match_id,stage,h_goal,a_goal,home_team,predict_result,away_team,draw,lose,win,actual_res
418,7264,27,1.0,0.0,Tottenham Hotspur,win,Arsenal,0.258878,0.272615,0.468507,win
419,7278,28,2.0,1.0,Manchester United,win,Chelsea,0.263224,0.295409,0.441367,win
420,7277,28,0.0,1.0,Crystal Palace,lose,Tottenham Hotspur,0.272214,0.414405,0.313382,lose
421,7276,28,1.0,2.0,West Bromwich Albion,win,Huddersfield,0.245164,0.226139,0.528697,lose
422,7275,28,1.0,0.0,Watford,win,Everton,0.268176,0.334390,0.397433,win
423,7273,28,1.0,1.0,Leicester City,win,Stoke City,0.256898,0.268341,0.474762,draw
424,7272,28,1.0,1.0,Burnley,win,Southampton,0.268909,0.323157,0.407935,draw
425,7271,28,4.0,1.0,Brighton,win,Swansea City,0.264298,0.300176,0.435526,win
426,7270,28,2.0,2.0,Bournemouth,win,Newcastle United,0.261751,0.292190,0.446059,draw
427,7274,28,4.0,1.0,Liverpool,win,West Ham United,0.248335,0.238980,0.512685,win


## Test data result from weight models

In [37]:
compare_results_3 = compare_results.copy()

# weight data
compare_results_3['draw'] = compare_results['draw'] * model_weight + compare_results_2['draw'] * (1 - model_weight)
compare_results_3['lose'] = compare_results['lose'] * model_weight + compare_results_2['lose'] * (1 - model_weight)
compare_results_3['win'] = compare_results['win'] * model_weight + compare_results_2['win'] * (1 - model_weight)
compare_results_3['predict_result'] = compare_results_3.apply(predict_home_result, axis=1)
compare_results_3.tail(10)

,match_id,stage,h_goal,a_goal,home_team,predict_result,away_team,draw,lose,win,actual_res
418,7264,27,1.0,0.0,Tottenham Hotspur,win,Arsenal,0.229050,0.358969,0.411981,win
419,7278,28,2.0,1.0,Manchester United,win,Chelsea,0.197889,0.357705,0.444406,win
420,7277,28,0.0,1.0,Crystal Palace,lose,Tottenham Hotspur,0.159717,0.642391,0.197891,lose
421,7276,28,1.0,2.0,West Bromwich Albion,lose,Huddersfield,0.308442,0.350258,0.341300,lose
422,7275,28,1.0,0.0,Watford,draw,Everton,0.353869,0.346133,0.299998,win
423,7273,28,1.0,1.0,Leicester City,win,Stoke City,0.298757,0.183526,0.517717,draw
424,7272,28,1.0,1.0,Burnley,win,Southampton,0.248475,0.332912,0.418613,draw
425,7271,28,4.0,1.0,Brighton,win,Swansea City,0.261995,0.312826,0.425179,win
426,7270,28,2.0,2.0,Bournemouth,win,Newcastle United,0.219438,0.282283,0.498279,draw
427,7274,28,4.0,1.0,Liverpool,win,West Ham United,0.250699,0.144751,0.604550,win


## Accuracy score of each model

In [38]:
# evaluate accuracy of prediction
print('Accuracy of prediction (old match) = {:.2f}%'.format(accuracy_score(compare_results['actual_res'], compare_results['predict_result']) * 100))

print('Accuracy of prediction (team rating) = {:.2f}%'.format(accuracy_score(compare_results_2['actual_res'], compare_results_2['predict_result']) * 100))

Accuracy of prediction (old match) = 58.41%
Accuracy of prediction (team rating) = 57.71%


In [39]:
# targets # ผลการแข่งขันจริงๆ
print('Total accuracy of prediction = {:.2f}%'.format(accuracy_score(compare_results_3['actual_res'], compare_results_3['predict_result']) * 100))

Total accuracy of prediction = 59.11%


## Predict next week from 2 model

#### From first model

In [40]:
compare_results_1_1 = model_test_matches_1[['match_id', 'stage', 'home_team_goal', 
                                    'away_team_goal', 'home_team', 'away_team']].copy()
compare_results_1_1.rename(columns={'home_team_goal':'h_goal', 'away_team_goal':'a_goal'}, inplace=True)
compare_results_1_1 = pd.concat([compare_results_1_1, predicted_table_1], axis=1)
compare_results_1_1 = compare_results_1_1[['match_id', 'stage', 'h_goal', 'a_goal', 'home_team', 'predict_res', 'away_team','draw', 'lose', 'win', 'actual_res']]
compare_results_1_1 = compare_results_1_1.rename(columns={'predict_res': 'predict_result'})
compare_results_1_1

,match_id,stage,h_goal,a_goal,home_team,predict_result,away_team,draw,lose,win,actual_res
0,7284,29,NaN,NaN,Manchester City,win,Chelsea,0.162514,0.288292,0.549194,draw
1,7286,29,NaN,NaN,Swansea City,win,West Ham United,0.299066,0.266719,0.434215,draw
2,7285,29,NaN,NaN,Southampton,win,Stoke City,0.287815,0.190106,0.522079,draw
3,7283,29,NaN,NaN,Liverpool,win,Newcastle United,0.171096,0.092705,0.736199,draw
4,7287,29,NaN,NaN,Tottenham Hotspur,win,Huddersfield,0.278775,0.232009,0.489216,draw
5,7281,29,NaN,NaN,Crystal Palace,lose,Manchester United,0.132910,0.694523,0.172566,draw
6,7280,29,NaN,NaN,Burnley,win,Everton,0.323600,0.336643,0.339757,draw
7,7279,29,NaN,NaN,Brighton,lose,Arsenal,0.274790,0.531308,0.193902,draw
8,7282,29,NaN,NaN,Leicester City,win,Bournemouth,0.213534,0.203100,0.583366,draw
9,7288,29,NaN,NaN,Watford,win,West Bromwich Albion,0.352044,0.269413,0.378543,draw


#### From second model

In [41]:
compare_results_2_1 = model_test_matches_1[['match_id', 'stage', 'home_team_goal', 
                                    'away_team_goal', 'home_team', 'away_team']].copy()
compare_results_2_1.rename(columns={'home_team_goal':'h_goal', 'away_team_goal':'a_goal'}, inplace=True)
compare_results_2_1 = pd.concat([compare_results_2_1, predicted_table_2_1], axis=1)
compare_results_2_1 = compare_results_2_1[['match_id', 'stage', 'h_goal', 'a_goal', 'home_team', 'predict_res', 'away_team','draw', 'lose', 'win', 'actual_res']]
compare_results_2_1 = compare_results_2_1.rename(columns={'predict_res': 'predict_result'})
compare_results_2_1

,match_id,stage,h_goal,a_goal,home_team,predict_result,away_team,draw,lose,win,actual_res
0,7284,29,NaN,NaN,Manchester City,win,Chelsea,0.261194,0.280002,0.458803,draw
1,7286,29,NaN,NaN,Swansea City,win,West Ham United,0.270668,0.353769,0.375563,draw
2,7285,29,NaN,NaN,Southampton,win,Stoke City,0.260327,0.282243,0.457430,draw
3,7283,29,NaN,NaN,Liverpool,win,Newcastle United,0.231115,0.191718,0.577167,draw
4,7287,29,NaN,NaN,Tottenham Hotspur,win,Huddersfield,0.206614,0.141726,0.651660,draw
5,7281,29,NaN,NaN,Crystal Palace,lose,Manchester United,0.272003,0.424605,0.303392,draw
6,7280,29,NaN,NaN,Burnley,win,Everton,0.272824,0.362318,0.364858,draw
7,7279,29,NaN,NaN,Brighton,lose,Arsenal,0.273445,0.442397,0.284158,draw
8,7282,29,NaN,NaN,Leicester City,win,Bournemouth,0.250231,0.241943,0.507826,draw
9,7288,29,NaN,NaN,Watford,win,West Bromwich Albion,0.262508,0.296195,0.441297,draw


## Last result

In [42]:
compare_results_3_1 = compare_results_1_1.copy()

# weight data
compare_results_3_1['draw'] = compare_results_1_1['draw'] * model_weight + compare_results_2_1['draw'] * (1 - model_weight)
compare_results_3_1['lose'] = compare_results_1_1['lose'] * model_weight + compare_results_2_1['lose'] * (1 - model_weight)
compare_results_3_1['win'] = compare_results_1_1['win'] * model_weight + compare_results_2_1['win'] * (1 - model_weight)
compare_results_3_1['predict_result'] = compare_results_3_1.apply(predict_home_result, axis=1)
compare_results_3_1.drop('actual_res', axis=1, inplace=True)
compare_results_3_1

,match_id,stage,h_goal,a_goal,home_team,predict_result,away_team,draw,lose,win
0,7284,29,NaN,NaN,Manchester City,win,Chelsea,0.182250,0.286634,0.531116
1,7286,29,NaN,NaN,Swansea City,win,West Ham United,0.293386,0.284129,0.422484
2,7285,29,NaN,NaN,Southampton,win,Stoke City,0.282318,0.208533,0.509149
3,7283,29,NaN,NaN,Liverpool,win,Newcastle United,0.183100,0.112507,0.704393
4,7287,29,NaN,NaN,Tottenham Hotspur,win,Huddersfield,0.264342,0.213953,0.521705
5,7281,29,NaN,NaN,Crystal Palace,lose,Manchester United,0.160729,0.640540,0.198732
6,7280,29,NaN,NaN,Burnley,win,Everton,0.313445,0.341778,0.344777
7,7279,29,NaN,NaN,Brighton,lose,Arsenal,0.274521,0.513526,0.211953
8,7282,29,NaN,NaN,Leicester City,win,Bournemouth,0.220873,0.210869,0.568258
9,7288,29,NaN,NaN,Watford,win,West Bromwich Albion,0.334137,0.274770,0.391094
